<a href="https://colab.research.google.com/github/dasaparnabme07/Reinforcement-Learning-Based-Prosthetic-Hand-Control/blob/main/RL_Based_Prosthetic_Hand_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
from gym import spaces
import matplotlib.pyplot as plt

In [4]:
# Prosthetic hand environment (Actor-Critic)
class ProstheticHandEnv(gym.Env):
    def __init__(self, emg_features, target_actions):
        super(ProstheticHandEnv, self).__init__()
        self.emg_features = emg_features
        self.target_actions = target_actions
        self.num_samples = len(emg_features)

        # Define observation and action spaces
        self.observation_space = spaces.Box(low=-1.0, high=1.0, shape=(emg_features.shape[1],), dtype=np.float32)
        self.action_space = spaces.Discrete(len(np.unique(target_actions)))
        self.current_index = 0
        def reset(self):
          self.current_index = np.random.randint(0, self.num_samples)#Reset environment
          return self.emg_features[self.current_index]

    def step(self, action):
        #Apply action, compute reward, and update state
        target_action = self.target_actions[self.current_index]
        reward = 1.0 if action == target_action else -1.0
        self.current_index = (self.current_index + 1) % self.num_samples
        done = self.current_index == 0
        return self.emg_features[self.current_index], reward, done, {}

    def render(self, mode='human'):
        print(f"Current step: {self.current_index}")

In [6]:
# Define the Actor and Critic networks
class Actor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Actor, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.fc(x)

class Critic(nn.Module):
    def __init__(self, input_dim):
        super(Critic, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.fc(x)

In [7]:
# Actor-Critic agent
class ActorCriticAgent:
    def __init__(self, input_dim, action_dim, lr=0.001, gamma=0.99):
        self.actor = Actor(input_dim, action_dim)
        self.critic = Critic(input_dim)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr)
        self.gamma = gamma
        self.loss_fn = nn.MSELoss()

    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        probs = self.actor(state_tensor)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        return action.item(), dist.log_prob(action)

    def update(self, state, reward, next_state, log_prob, done):
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)

        # Calculate targets
        target = reward + (1 - done) * self.gamma * self.critic(next_state_tensor).item()
        target = torch.tensor(target).unsqueeze(0)

        # Critic loss
        value = self.critic(state_tensor)
        critic_loss = self.loss_fn(value, target)

        # Actor loss
        advantage = target - value.detach()
        actor_loss = -log_prob * advantage

        # Update actor and critic
        self.actor_optimizer.zero_grad()
        self.critic_optimizer.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        self.actor_optimizer.step()
        self.critic_optimizer.step()

In [11]:
# Load dataset (replace 'emg_data.csv' with your file)
import scipy.io
import pandas as pd
data= scipy.io.loadmat('/content/S1_A1_E1.mat')
emg_signals = data.iloc[:, :-1].values  # EMG channels
target_actions = data.iloc[:, -1].values  # Target actions

# Extract features
sampling_rate = 1000  # Adjust based on your dataset
emg_features = np.array([extract_features(signal, sampling_rate) for signal in emg_signals])

AttributeError: 'dict' object has no attribute 'iloc'